In [1]:
# Model parameters
vocab_size = 92
n_embd = 384
block_size = 256
batch_size = 64
dropout=0.2
n_head = 4
n_layer = 4

device = 'cpu' 

In [2]:
with open('sh.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line for line in lines if not line.strip().startswith(tuple('0123456789*'))]

    
    

In [3]:
text = ''.join(lines)
print(text)

     To Sherlock Holmes she is always the woman. I have seldom heard him
     mention her under any other name. In his eyes she eclipses and
     predominates the whole of her sex. It was not that he felt any
     emotion akin to love for Irene Adler. All emotions, and that one
     particularly, were abhorrent to his cold, precise but admirably
     balanced mind. He was, I take it, the most perfect reasoning and
     observing machine that the world has seen, but as a lover he would
     have placed himself in a false position. He never spoke of the softer
     passions, save with a gibe and a sneer. They were admirable things
     for the observer--excellent for drawing the veil from men's motives
     and actions. But for the trained reasoner to admit such intrusions
     into his own delicate and finely adjusted temperament was to
     introduce a distracting factor which might throw a doubt upon all his
     mental results. Grit in a sensitive instrument, or a crack in one of
   

In [4]:
vocab = sorted(list(set(text)))
print(vocab)
vocab_size = len(vocab)
print('Vocabulary size:', len(vocab))

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '©', '½', 'Â', 'Ã']
Vocabulary size: 81


In [5]:
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for i, ch in enumerate(vocab)}
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join([itos[i] for i in l])
l = encode('Chicken')
print(decode(l))

# use tiktoken

Chicken


In [6]:
import torch 
#import tiktoken




In [7]:
#enc = tiktoken.get_encoding('gpt2')
#print(enc.encode('Chicken'))

In [8]:
#print(enc.encode('something else'))

In [9]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

# not using tiktoken here

torch.Size([252100]) torch.int64
tensor([ 1,  1,  1,  1,  1, 43, 64,  1, 42, 57, 54, 67, 61, 64, 52, 60,  1, 31,
        64, 61, 62, 54, 68,  1, 68, 57, 54,  1, 58, 68,  1, 50, 61, 72, 50, 74,
        68,  1, 69, 57, 54,  1, 72, 64, 62, 50, 63, 10,  1, 32,  1, 57, 50, 71,
        54,  1, 68, 54, 61, 53, 64, 62,  1, 57, 54, 50, 67, 53,  1, 57, 58, 62,
         0,  1,  1,  1,  1,  1, 62, 54, 63, 69, 58, 64, 63,  1, 57, 54, 67,  1,
        70, 63, 53, 54, 67,  1, 50, 63, 74,  1])


In [10]:
n = int(0.8*len(data))
train = data[:n]
test = data[n:]


In [11]:


def get_batch(split):
    data = train if split == "train" else test
    index = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in index])
    y = torch.stack([data[i+1:i+block_size+1] for i in index])
    return x,y

xb,yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb)

torch.Size([64, 256])
tensor([[51, 54, 54,  ...,  1, 53, 64],
        [54,  1, 57,  ...,  1, 67, 54],
        [58, 68,  1,  ...,  0,  1,  1],
        ...,
        [69,  1, 66,  ..., 69, 57, 50],
        [74,  1, 57,  ..., 51, 74,  1],
        [ 1, 57, 58,  ...,  1, 68, 54]])
tensor([[54, 54, 63,  ..., 53, 64, 72],
        [ 1, 57, 50,  ..., 67, 54, 53],
        [68,  1, 51,  ...,  1,  1,  1],
        ...,
        [ 1, 66, 70,  ..., 57, 50, 69],
        [ 1, 57, 70,  ..., 74,  1, 50],
        [57, 58, 68,  ..., 68, 54, 67]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout= nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        v = self.value(x)
        out  = wei @ v

        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

class MultiHead(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout= nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHead(n_head, head_size)
        self.ffw = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffw(self.ln2(x))
        return x


# class LayerNorm:
#     def __init__(self, dim, eps = 1e-5, momentum = 0.1):
#         self.eps = eps
#         self.momentum = momentum
#         self.gamma = torch.ones(dim)
#         self.beta = torch.zeros(dim)

#     def __call__(self,x):
#         xmean = x.mean(1, keepdim=True)
#         xvar = x.var(1, keepdim=True)
#         xhat = (xmean-xvar)/torch.sqrt(xvar + self.eps)
#         self.out = self.gamma*xhat + self.beta
#         return self.out
    
#     def parameters(self):
#         return [self.gamma, self.beta]


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embeddin_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd)
        )
        # self.sa_heads = MultiHead(4,n_embd//4)
        # self.ffw = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        tokenembd = self.token_embedding_table(idx)
        posemb = self.position_embeddin_table(torch.arange(T, device = idx.device))
        x = tokenembd + posemb
        x = self.blocks(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits,loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next), dim = 1)
        return idx




    
m = BigramLanguageModel().to(device)
out, loss = m(xb,yb)
print(out.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([16384, 81])
tensor(4.5484, grad_fn=<NllLossBackward0>)

bM?O
6gdi,A!"2G3
QaMsean8(`JA1)U7qiezuj£ElrbA"©,zNOAo&2©7KPMCGipxwqEc5'kNJP2Âu8"UP-
ZJa"Yky:T8"hQEiF


In [59]:


optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

for steps in range(200):
    xb,yb = get_batch("train")

    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

0.6327095627784729


In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long).to(device)

output_idx = m.generate(idx, max_new_tokens=1000)

print(decode(output_idx[0].tolist()))


     thanking long and excitement and extrenedings the stronges and the
     obviousin of his word glances. Mr. McCarthy rellapping up to the
     hands upon half a gold and door?" he asked.

     "Yes, gryman. You may know, Mr. Hosmer Angel. You are you very can recomplimber about my
     really over hangedrom anything what
     then? That do you indeas it."

     "Then I shall no uncome to go to you, as I just to go; and watch you,
     rexample how observed you for my father daring his wife with a left
     her loured trays step in her judge upon his could breathing
     to harness to. The ft beaten tastion her from the windot of
     the case is as we en bears Mr. Hosmer Angel."

     Holmes had bank near she effore and upon his chook analy. Holmes
     and a few lock his peck and sadvantage of a who frequest, and his
     cuniquential sy-landed now, is observer, he gauther, so
     veright very flimbs to seven newhy the scaness amoky betters indeped
     blacked meself-coat. Andl

In [61]:
torch.save(m.state_dict(), 'farturConanDoyle.pth')

In [ ]:
m.eval()

with torch.no_grad():
    xb, yb = get_batch("test")  
    logits, test_loss = m(xb, yb)

print(f"Test Loss: {test_loss.item():.4f}")

m.train()

Test Loss: 1.9672


BigramLanguageModel(
  (token_embedding_table): Embedding(81, 384)
  (position_embeddin_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHead(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=384, out_features=96, bias=False)
            (query): Linear(in_features=384, out_features=96, bias=False)
            (value): Linear(in_features=384, out_features=96, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffw): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
 